In [10]:
import pandas as pd
import re

In [60]:
years = ['1999', '2001', '2003', '2005', '2007',
         '2009', '2011', '2013', '2015', '2017', 
         '2019', '2000', '2002', '2004', '2006',
         '2008', '2010', '2012', '2014', '2016', 
         '2018']

THERE'S SOME WEIRD STUFF LIKE #=72 IN 2018...GO BACK AND FIX THIS...

In [136]:
df_dict = {}
for eachYear in years: 
    file_2019 = open('./raw_wiki_data/%s.txt' % str(eachYear), 'r')

    current_case = 0
    justice_info = {}
    justice_df = pd.DataFrame()
    for each_line in file_2019:
        ###### FIX THIS PART TO FIX THE JUSTICES
        # Getting the justices:
        if 'SCOTUS-termlist-start' in each_line:
            justices_info = each_line.strip().strip('}}').split('|')[1:]
            justices_info = [item.lower().replace(' ', '') for item in justices_info]
            justice_df = pd.DataFrame(columns=justices_info)

        # If you hit this, that means you're on a new case:
        if '#=' in each_line:
            # Updating the iterator:
            previous_case = current_case
            current_case += 1
            # If you don't have any justice info yet, continue
            if len(justice_info.keys()) == 0:
                continue
            # Otherwise, make a df out of that info:
            temp_df = pd.DataFrame.from_dict(justice_info)
            justice_df = justice_df.append(temp_df)
            # Reset the dict:
            justice_info = {}

        # If there's no justice info, fuggetaboutit
        if '|justice' not in each_line:
            continue

        ###### FIX THIS PART TO FIX THE OPINIONS
        # Otherwise...
        else:
            # Start by getting the justice:
            justice_list = each_line.split('--')
            justice = justice_list[1].replace(' not yet appointed', '').replace('retired', '').replace(' ', '').strip()
            # Get the opinion:
            opinion = justice_list[0].split('=')[1].split(' |')[0].replace(' not yet appointed', '').replace('retired', '').replace(' ', '').strip()
            justice_info[justice.lower()] = [opinion]

    justice_df.to_excel('./parsed_wiki_data/%s.xlsx' % str(eachYear))
    df_dict[eachYear] = justice_df
#     if eachYear == '2008':
#         break

Right now, I'm doing splitting...instead, I'm going to get all possible strings, and pattern match:

In [154]:
all_combined = []
for each_year in df_dict.keys():
    for each_column in df_dict[each_year]:
        temp_column_val = list(set([re.sub(r'\d+', '', item) for item in list(set(df_dict[each_year][each_column]))]))
        all_combined = all_combined + temp_column_val

In [156]:
# Cleaning up the all_combined, and using that to determine what the judges did:
print(len(all_combined))
all_combined = list(set(all_combined))
cleaned_all_combined_v1 = [re.sub(r'\d+', '', item) for item in all_combined]
cleaned_all_combined_v2 = [item.split('|')[0] for item in cleaned_all_combined_v1]
cleaned_all_combined_v3 = list(set(cleaned_all_combined_v2))
print(len(cleaned_all_combined_v3))

29
19


In [157]:
cleaned_all_combined_v3

['didnotparticipate',
 'concurrencedissentwithoutopinion',
 'partjoinmajority',
 'majority',
 'concurrence',
 'partjoinconcurrence',
 'joinplurality',
 'partjoinplurality',
 'dissentwithoutopinion',
 'dissent',
 'concurrencedissent',
 'concurrencewithoutopinion',
 'joinconcurrencedissent',
 'joinmajority',
 'plurality',
 'joinconcurrence',
 'partjoindissent',
 'partjoinconcurrencedissent',
 'joindissent']